In [1]:
from czitools import pylibczirw_metadata as czimd
from czitools import pylibczirw_tools
from pylibCZIrw import czi as pyczi
import os
from tqdm.contrib.itertools import product
import numpy as np
from pathlib import Path

In [2]:
# get the defulat directory
defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")

filepath = os.path.join(defaultdir, "CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi")

# return 6D array with dimension order STZCYXA
array6d, mdata, dim_string6d,  = pylibczirw_tools.read_6darray(filepath,
                                                               output_order="STZCYX",
                                                               chunks_auto=False,
                                                               output_dask=False,
                                                               remove_Adim=False)

No Objective Name : 'Name'
No Objective ID : 'Id'
DetectorID not found : 'Id'
DetectorName not found : 'Name'
DetectorType not found : 'Type'
Microscope System Name not found : 'System'
No Scene or Well information found.


  0%|          | 0/30 [00:00<?, ?it/s]

## ZSTD compression

* zstd0 - You can specify the parameters of it like "explicitlevel" between -131072 and 22. (e.g. "zstd0:ExplicitLevel=10")
* zstd1 - You can specify the parameters of it like "explicitlevel" between -131072 and 22 or "preprocess" - only have HiLoByteUnpack.
  * This is only done for zstd1 and for pixel types gray16 and Bgr48. (e.g. "zstd1:ExplicitLevel=10" or "zstd1:Preprocess=HiLoByteUnpack")

To read more -: https://facebook.github.io/zstd/

In [3]:
# define the compression parameters

#cmp = "zstd0:ExplicitLevel=3"
#cmp = "zstd1:ExplicitLevel=10"
#cmp = "zstd1:Preprocess=HiLoByteUnpack;ExplicitLevel=3"
cmp = "zstd1:Preprocess=HiLoByteUnpack;Level=Best"

filepath_comp = filepath[:-4] + "_comp.czi"

In [4]:
# write the same CZI image back to disk but this time with using ZST compression

# create a CZI to write to
with pyczi.create_czi(filepath_comp, exist_ok=True) as czidoc_w:

    # iterate over all 2D planes
    for s, t, z, c in product(range(array6d.shape[0]),
                              range(array6d.shape[1]),
                              range(array6d.shape[2]),
                              range(array6d.shape[3])):

        # writing the image with compression
        czidoc_w.write(
            data=array6d[s, t, z, c, ...],
            plane={"C": c, "T": t, "Z": z},
            scene=s,
            compression_options=cmp,  # valid for individual planes
        )

  0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
size = os.path.getsize(filepath)/1024.0**2
print(f"Size of the uncompressed CZI file : {np.round(size, 2)} MB ")

size_comp = os.path.getsize(filepath_comp)/1024.0**2
print(f"Size of the compressed CZI file   : {np.round(size_comp, 2)} MB ")

print(f"ZSTD Compression Ratio: {np.round(size/size_comp, 2)}")

Size of the uncompressed CZI file : 2.88 MB 
Size of the compressed CZI file   : 1.28 MB 
ZSTD Compression Ratio: 2.25
